In [28]:
%load_ext autoreload
%autoreload 2

In [29]:
import models
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import plotly
import plotly.express as px
import pandas as pd
import geopandas as gpd
import mscthesis

In [30]:
geotweets = mscthesis.read_geotweets('sweden').set_index('userid')

In [31]:
geotweets.groupby('userid').size()

userid
5616                  2130
5976                   287
8981                   941
32213                  413
392123                 617
                      ... 
764655612525473792      64
778179816889675777      36
782260642770288640      47
790301765744951296      54
794973407964229636     856
Length: 4056, dtype: int64

In [62]:
usertweets = geotweets.loc[5616]
home_location = usertweets[usertweets['label'] == 'home']

In [63]:
usertweets.groupby('region').head(1)

,region,createdat,tweetid,latitude,longitude,month,weekday,hourofday,timezone,ym,label,geometry
userid,,,,,,,,,,,,
5616,0,2015-05-07 15:12:52+00:00,596331871241379840,57.599221,18.436371,5,4,17,Europe/Stockholm,2015-05,other,POINT (18.43637 57.59922)
5616,1,2013-11-25 18:59:40+00:00,405048155831799808,59.442985,17.952117,11,1,19,Europe/Stockholm,2013-11,other,POINT (17.95212 59.44299)
5616,2,2013-11-01 04:41:30+00:00,396134884584194048,59.652696,17.930374,11,5,5,Europe/Stockholm,2013-11,other,POINT (17.93037 59.65270)
5616,4,2013-03-17 16:46:39+00:00,313330551073419264,59.452536,17.930460,3,0,17,Europe/Stockholm,2013-03,other,POINT (17.93046 59.45254)
5616,5,2013-03-09 17:45:46+00:00,310446326309584896,59.426889,17.954336,3,6,18,Europe/Stockholm,2013-03,home,POINT (17.95434 59.42689)
...,...,...,...,...,...,...,...,...,...,...,...,...
5616,610,2017-08-19 16:46:01+00:00,898949180906917889,60.172845,18.185741,8,6,18,Europe/Stockholm,2017-08,other,POINT (18.18574 60.17285)
5616,611,2017-08-27 12:58:26+00:00,901791011600519168,59.380944,18.180836,8,0,14,Europe/Stockholm,2017-08,other,POINT (18.18084 59.38094)
5616,612,2017-08-27 13:32:27+00:00,901799574330896384,59.362910,18.152143,8,0,15,Europe/Stockholm,2017-08,other,POINT (18.15214 59.36291)


In [64]:
samples = []
def save_sample(uid, day, timeslot, d, sample):
    s = [uid, day, timeslot]
    if sample[0] == 'region':
        s = s + ['region', sample[1], sample[2], sample[3]]
    else:
        s = s + ['point', -1, sample[1], sample[2]]
    d.append(s)

i = 0
for uid in geotweets.index.unique():
    usertweets = geotweets.loc[uid]
    m = models.SongModel(p=0.66, gamma=0.6, zipf=-1.7)
    m.fit(usertweets)
    home_location = usertweets[usertweets['label'] == 'home']
    for day in range(7*20):
        prev = ("region", home_location.iloc[0]['region'], home_location.iloc[0]['latitude'], home_location.iloc[0]['longitude'])
        save_sample(uid, day, 0, samples, prev)
        for timeslot in range(3):
            prev = m.sample(prev)
            save_sample(uid, day, timeslot+1, samples, prev)
    i += 1
    if i % 250 == 0:
        print("done with", i)

5616
5976
8981
32213
392123
618063
625323
633523
641033
646913
666903
756227
792370
806842
1047261
1161411
1227871
1503691
1723131
1946401
2038581
2098971
2151021
2839211
2940561
3835271
3899811
3942101
4002741
4763391
5040331
5230731
5309462
5430382
5612352
5658712
5687962
5728752
5747132
5828272
5872222
6007132
6732492
6733372
7135202
7184952
7247412
7344452
7459272
7468072
7620852
7646022
7877902
7908272
8025132
8026912
8223662
8505972
8608182
9344772
9398042
9563622
9568442
9575752
9646812
9679552
9819332
9877492
10566262
10626432
10939792
11294332
11321802
11351412
11773412
11981172
12020922
12170292
12243562
12259782
12275732
12278352
12890272
13014832
13387012
13719002
13766752
13943582
13959762
14047767
14056639
14056802
14079762
14106909
14127233
14139031
14151819
14153467
14157752
14162766
14224582
14273947
14277184
14282146
14299896
14333891
14340780
14369301
14377557
14406249
14406897
14407909
14446419
14451219
14452905
14460499
14510363
14556856
14558773
14568707
14602106


46498061
46623028
46683150
46730659
46752529
46764966
46883238
46892186
46903494
47074156
47081457
47087320
47202594
47299693
47313630
47326582
47366950
47408424
47599484
47609264
47630286
47669762
47917771
47917778
47922431
47939696
48029037
48137641
48291553
48392637
48668084
48704949
48820406
49053587
49278776
49372238
49442520
49560384
49573917
49578426
49591163
49593331
49603021
49610970
49751942
49758112
49916918
49931698
49933770
49959063
49977734
50052289
50081440
50218527
50273625
50274011
50382174
50444314
50578678
50581116
50593354
51084466
51313476
51400403
52145467
52148944
52190770
52430725
52515166
52666132
52674342
52708217
52748327
53035938
53185597
53342277
53375274
53502188
53544992
53792305
53888435
54024412
54183261
54216936
54218847
54806689
54873587
54901213
55004612
55164395
55222246
55859399
56097600
56174901
56321125
56775773
56820460
57076623
57318279
57352680
57405611
57424125
57509718
57625159
57629053
57973288
58018764
58036135
58486598
58543016
58577113
5

190160570
190222138
190553074
191015664
191061769
191378721
191956388
192501598
192584907
192601855
192685590
192860728
193272275
193312988
193332266
193434381
193444352
193675136
193729593
193753775
193840906
193860919
193988694
194026341
194282445
194333538
194614807
195266336
195319619
195705459
195809033
195887660
196310151
196604171
196773575
196806894
197367761
197375850
197610297
197721814
197887676
198086470
198197830
198248715
198616878
198776615
198998535
199417956
199794508
200117296
200240159
200544721
200609775
200849662
201236940
201419910
201722164
202200776
202247883
202275894
202298925
202347378
202364821
202596834
203655297
203910492
204799808
205205058
205599088
205758464
205990866
206140202
206254283
206693335
206860150
207469874
208387755
208986309
209560789
209618294
209981408
210159820
210251384
210413528
210644805
210920939
211094605
211176343
211326901
211353359
211525139
211804943
211905087
211963094
212027541
212355707
212405648
212879331
212896119
213483236


397231933
397417195
397599429
397796573
397854615
397995328
398518183
398990688
399368019
399671325
399985022
400179130
400278966
400318756
400568928
400673907
400797298
401877024
402030035
402168358
402953220
403052132
403068780
403669688
403684327
403687954
403950322
404118675
404315916
405779973
406140358
406253644
406986706
407076657
407132266
407188419
407728133
407770195
408374949
408489532
409102763
409108505
409122830
409277778
409881668
409983247
410063535
410087506
410209948
411355822
411394945
411524038
411588241
411591812
411599643
411605714
411891396
412213554
412934114
413078913
413836619
413861811
413928633
413974993
414023322
414071488
414430543
414640188
414817352
415359219
415439118
415551982
416128416
416396329
416517809
416526381
417217786
418109058
418155090
418393454
418489309
418603582
419042134
419359558
419659192
420093583
420103723
420161947
420298715
420304894
421013817
421331907
421915743
422038152
422207465
422960476
423319679
423588016
423820402
424197987


1023530131
1024498897
1024872276
1026646940
1028828394
1030149210
1030643810
1030906284
1032415417
1032886015
1034028787
1034980746
1035669139
1037092274
1037127757
1039783262
1041575089
1043378515
1047191610
1049061396
1054904851
1055973727
1057960934
1058723288
1062886562
1067666917
1071235146
1072142629
1075680798
1077693362
1087209600
1088376336
1090037772
1091496835
1091678174
1094762286
1095324878
1097855689
1100074382
1100413104
1101852397
1103339688
1108533092
1114218799
1114302624
1117854601
1117926378
1121309215
1124579941
1128349273
1129200361
1130811182
1134482876
1134772568
1135528142
1136850691
1139212080
1143254132
1143405894
1143713726
1144478990
1144821403
1144851294
1146174085
1147876015
1147921586
1148873208
1149548186
1151301517
1151918712
1156894950
1157078744
1162442419
1164127520
1164584964
1165033514
1165423628
1167463704
1168705284
1169731676
1173530340
1174736112
1180373143
1181526007
1182462817
1183511996
1189118802
1198398654
1203401162
1208490470
1213188374

ValueError: Grouper for 'region' not 1-dimensional

In [65]:
samplesx = pd.DataFrame(
    samples,
    columns=['userid', 'day', 'timeslot', 'kind', 'region', 'latitude', 'longitude'],
).set_index('userid')

In [68]:
samplesx.to_csv('./song-model-trips.csv')